In [ ]:
#!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu

In [ ]:
#conda install pytorch torchvision torchaudio cpuonly -c pytorch

In [ ]:
#!pip install cvzone
#!pip install torchvision==0.17.2
#!pip install face_recognition 
#conda install -c conda-forge dlib
#!pip install glasses-detector
#!pip install cmake
#!pip install --upgrade yagmail
#app pass for my device: cnmq kztq ofqe hjfu, edva cmrv oyfq nvwq

## Importing Necessary Libraries

In [1]:
import pandas as pd
import face_recognition
from ultralytics import YOLO
import cv2
import cvzone
import math
import os
import dlib
import numpy as np

## Implementing Email Reporting

In [6]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def test_email():
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587 # TLS port: 587 #SSL port: 465
    sender_email = 'mariamshalaby192@gmail.com'
    password = 'edva cmrv oyfq nvwq'
    receiver_email = 'mariamshalaby135@gmail.com'
    #'mohamed.mosa@lafarge.com'

    subject = "Test Email"
    body = "This is a test email sent using smtplib."

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.set_debuglevel(1)  # Enable debug output
            server.starttls()  # Upgrade the connection to secure
            server.login(sender_email, password)
            server.send_message(msg)
            print("Test email sent successfully!")
    except Exception as e:
        print(f"Failed to send test email: {e}")

test_email()


send: 'ehlo [192.168.133.19]\r\n'
reply: b'250-smtp.gmail.com at your service, [196.136.22.68]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [196.136.22.68]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [192.168.133.19]\r\n'
reply: b'250-smtp.gmail.com at your service, [196.136.22.68]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at y

Test email sent successfully!


reply: b'250 2.0.0 OK  1724051237 ffacd0b85a97d-371898aa393sm9592383f8f.90 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1724051237 ffacd0b85a97d-371898aa393sm9592383f8f.90 - gsmtp'
data: (250, b'2.0.0 OK  1724051237 ffacd0b85a97d-371898aa393sm9592383f8f.90 - gsmtp')
send: 'QUIT\r\n'
reply: b'221 2.0.0 closing connection ffacd0b85a97d-371898aa393sm9592383f8f.90 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection ffacd0b85a97d-371898aa393sm9592383f8f.90 - gsmtp'


In [7]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_ppe_violation_email(name, department, ppe_status):
    # Email settings
    sender_email = 'mariamshalaby192@gmail.com'
    password = 'edva cmrv oyfq nvwq'  # Make sure this is your actual app password for Gmail
    receiver_email = "mariamshalaby135@gmail.com"
    
    # Create the email content
    subject = f"PPE Violation Alert: {name}"
    body = f"""
    Dear Safety Officer,

    The following worker has been detected not wearing required PPEs:

    Name: {name}
    Department: {department}
    Missing PPEs: {", ".join(ppe_status)}

    Please take the necessary actions.

    Best regards,
    Safety Monitoring System
    """
    
    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    
    # Add body to email
    message.attach(MIMEText(body, "plain"))
    
    # Log in to server and send the email
    try:
        # Use the correct Gmail SMTP server and port
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()  # Secure the connection
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message.as_string())
        server.quit()
        print(f"Email sent to {receiver_email} for {name}")
    except Exception as e:
        print(f"Failed to send email: {str(e)}")


In [8]:
# Path to the images directory
images_dir = './Dataset/'

# Load the dataset with worker information
workers_df = pd.read_csv('Dataset/workers_data.csv')

# Prepare known face encodings and metadata (name and department)
known_face_encodings = []
known_face_metadata = []

# Load pre-trained face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Loop through each row in the DataFrame to find image paths
for index, row in workers_df.iterrows():
    image_file = row['Image']  # Assuming 'Image' column contains filenames
    image_path = os.path.join(images_dir, image_file)
    
    if os.path.isfile(image_path):  # Check if the image file exists
        worker_image = face_recognition.load_image_file(image_path)
        face_encodings = face_recognition.face_encodings(worker_image)
        
        if face_encodings:  # Check if at least one face encoding was found
            known_face_encodings.append(face_encodings[0])
            known_face_metadata.append({
                "name": row['Name'],  # Assuming 'Name' and 'Department' are columns in workers_df
                "department": row['Department']
            })
    else:
        print(f"File not found: {image_path}")

# Initialize the video capture (webcam or video)
cap = cv2.VideoCapture(0)  # For Webcam
cap.set(3, 1280)  # Set width
cap.set(4, 720)   # Set height

# Load the PPE detection model
model = YOLO("ppe.pt")

# Define the class names for PPE items
classNames = ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Glasses', 'No-Glasses', 'Safety Vest']

while True:
    success, img = cap.read()
    if not success:
        break  # If there's an issue with the webcam, exit the loop

    # Detect PPE using the YOLO model
    results = model(img, stream=True)
    ppe_status = []

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            if conf > 0.5:
                if currentClass in ['NO-Hardhat', 'NO-Safety Vest', 'NO-Mask','No-Glasses']:
                    ppe_status.append(currentClass)

                cvzone.putTextRect(img, f'{currentClass} {conf}',
                                   (max(0, x1), max(35, y1)), scale=1, thickness=1, colorB=(0, 0, 255),
                                   colorT=(255, 255, 255), colorR=(0, 0, 255), offset=5)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

    # Face Recognition
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_img)
    face_encodings = face_recognition.face_encodings(rgb_img, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        department = ""

        if True in matches:
            first_match_index = matches.index(True)
            metadata = known_face_metadata[first_match_index]
            name = metadata["name"]
            department = metadata["department"]

        # Detect facial landmarks using dlib
        face_rect = dlib.rectangle(left, top, right, bottom)
        landmarks = predictor(rgb_img, face_rect)

        # Draw landmarks around the eyes
        glasses_detected = False
        for n in range(36, 48):  # Eye region landmarks
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(img, (x, y), 2, (255, 0, 0), -1)

        # Simple heuristic to determine if glasses are present
        # (e.g., distance between eye landmarks can give clues about glasses, though it's basic)
        eye_aspect_ratio = (landmarks.part(41).y - landmarks.part(37).y) / (landmarks.part(39).x - landmarks.part(36).x)
        if eye_aspect_ratio > 0.5:  # This threshold may need tuning
            glasses_detected = False

        if glasses_detected:
            cv2.putText(img, "Glasses Detected", (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        else:
            ppe_status.append("No-Glasses")

        # Display worker info
        display_text = f'Name: {name} | Dept: {department}'
        if ppe_status:
            display_text += f' | Missing PPE: {", ".join(ppe_status)}'

        # In the loop where you detect PPE violations, add:
        if ppe_status and name != "Unknown" :  # If PPE violations and face are detected
            send_ppe_violation_email(name, department, ppe_status)
            
        cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(img, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(img, display_text, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    
        
    # Display the video feed with detections
    cv2.imshow("Image", img)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 NO-Hardhat, 1 NO-Mask, 1 NO-Safety Vest, 1 Person, 2881.9ms
Speed: 416.6ms preprocess, 2881.9ms inference, 318.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 NO-Hardhat, 1 NO-Mask, 2 NO-Safety Vests, 1 Person, 656.2ms
Speed: 4.2ms preprocess, 656.2ms inference, 30.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 NO-Hardhat, 1 NO-Mask, 1 NO-Safety Vest, 1 Person, 737.1ms
Speed: 2.0ms preprocess, 737.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 NO-Hardhat, 1 NO-Mask, 2 NO-Safety Vests, 1 Person, 684.7ms
Speed: 4.2ms preprocess, 684.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Email sent to mariamshalaby135@gmail.com for Mariam Shalaby

0: 384x640 1 NO-Hardhat, 1 NO-Mask, 1 NO-Safety Vest, 1 Person, 788.0ms
Speed: 3.2ms preprocess, 788.0ms inference, 6.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 NO-Hardhat, 1 NO-Mask, 2 NO-Safety Vests, 1 Person, 840.2ms
Sp